<a href="https://colab.research.google.com/github/BorisOskolkov/MLcourse/blob/main/asng02/A02_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Made by Boris Oskolkov for Assignment03 (CS5783)

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import string
import pandas as pd
from IPython.display import Image

 --------- start of a first problem --------------

--------- end of a first problem --------------

--------- start of a second problem --------------

--------- end of a second problem --------------